In [23]:
# Set up the ArcGIS Project and relevant names
project = arcpy.mp.ArcGISProject("CURRENT")
layout_name = "TEST"  # Name of the layout
main_map_name = "Overview Map Frame"  # Name of the main map
geographic_location_field = "LICENCE_ID"  # Field name for Geographic Location

In [32]:
# Access the layout and the main map frame
layout = project.listLayouts(layout_name)[0]
main_map_frame = layout.listElements("MAPFRAME_ELEMENT", main_map_name)[0]
main_map = main_map_frame.map
print("Map Name: " + main_map.name)

Map Name: Overview


In [44]:
# Parameters for grid layout
inset_width = 4  # Width of each inset map frame
inset_height = 4  # Height of each inset map frame
rows = 8  # Number of rows for inset maps
cols = 1  # Number of columns for inset maps
spacing = 1  # Space between inset maps

# Get the layout's width and height
layout_width = layout.pageWidth
print("Width: " + str(layout_width))
layout_height = layout.pageHeight
print("Height: " + str(layout_height))

# Calculate the starting position for the grid of inset maps
start_x = 0.5  # Adjust this as needed for margins
start_y = layout_height - inset_height - 0.5  # Adjust this as needed for margins

Width: 32.0
Height: 36.0


In [34]:
# Loop through the unique values in the Geographic Location field
with arcpy.da.SearchCursor("FTAR 2025 Planting Blocks", [geographic_location_field]) as cursor:
    location_list = [row[0] for row in cursor]

print(location_list)
print()
location_list = list(dict.fromkeys(location_list))
print(f"Duplicates Removed: {location_list}")

['TA1616', 'TA0713', 'A93872', 'A93714', 'A95414', 'A95414', 'A95807', 'TA0470', 'A93714', 'A95807', 'A95807', 'TA0470', 'TA0470', 'A93873', 'A93873', 'TA0713', 'TA0713', 'TA0713', 'A93873', 'A95414', 'A95414', 'A95414']

Duplicates Removed: ['TA1616', 'TA0713', 'A93872', 'A93714', 'A95414', 'A95807', 'TA0470', 'A93873']


In [45]:
# Define a function to get the extent for a specific location
def get_extent_for_location(location):
    # Query the layer to find the extent for the given location
    layer = "FTAR 2025 Planting Blocks"  # Adjust if multiple layers or specific layer
    query = f"{geographic_location_field} = '{location}'"
    print("Query: " + query)
    extent_union = None  # This will store the combined extent
    
    # Use a search cursor to loop through all shapes with the matching location
    with arcpy.da.SearchCursor(layer, ["SHAPE@"], query) as cursor:
        for row in cursor:
            shape = row[0]
            extent = shape.extent
            
            if extent_union is None:
                # Initialize the extent_union with the first shape's extent
                extent_union = extent
            else:
                # Manually compute the union of extents by updating the min/max coordinates
                extent_union.XMin = min(extent_union.XMin, extent.XMin)
                extent_union.YMin = min(extent_union.YMin, extent.YMin)
                extent_union.XMax = max(extent_union.XMax, extent.XMax)
                extent_union.YMax = max(extent_union.YMax, extent.YMax)
    
    return extent_union

map_frames = layout.listElements("MAPFRAME_ELEMENT")

for i, location in enumerate(location_list):
    # Skip if there are no more pre-existing map frames
    if i >= len(map_frames):
        break
    
    # Get the current map frame
    map_frame = map_frames[i]
    print(map_frame.name)
    if map_frame.name == "Overview Map Frame":
        print("   ...Skiping Overview Map Frame")
        continue
    
    
    row = i // cols
    col = i % cols
    
    # Calculate the position for the map frame
    x_position = start_x + col * (inset_width + spacing)
    y_position = start_y - row * (inset_height + spacing)
    
    # Set the position and size for the map frame
    map_frame.elementPositionX = x_position
    map_frame.elementPositionY = y_position
    map_frame.elementWidth = inset_width
    map_frame.elementHeight = inset_height

    # Get the extent for the map frame based on the current location
    extent = get_extent_for_location(location)
    print(extent)
    print("")
    
    # Set the extent for the map frame
    map_frame.camera.setExtent(extent)
    
#     # Add a text element to label the inset map with the location
#     # Create the text element below the inset map
#     text_x_position = x_position
#     text_y_position = y_position - 0.5  # Adjust the spacing for text placement
    
#     # Create a new text element using arcpy.mp.TextElement
#     new_text_element = arcpy.mp.TextElement(layout)
#     new_text_element.text = f"{location}"
#     new_text_element.elementPositionX = text_x_position
#     new_text_element.elementPositionY = text_y_position
#     new_text_element.elementWidth = inset_width
#     new_text_element.elementHeight = 0.5

#     # Add the new text element to the layout
#     layout.addElement(new_text_element)

#     # Optionally, update other elements like the title
#     #title_element = layout.listElements("TEXT_ELEMENT", "Inset Title")[0]
#     #title_element.text = f"Inset for {location}"

# Save the project if changes were made
project.save()
print("   ...Script Complete")


Inset 1
Query: LICENCE_ID = 'TA1616'
1566689.8135 574485.5538 1567288.1983 575677.2355 NaN NaN NaN NaN
﻿
Inset 2
Query: LICENCE_ID = 'TA0713'
1560415.7401 576085.009 1563325.6182 581480.6143 NaN NaN NaN NaN
﻿
Inset 3
Query: LICENCE_ID = 'A93872'
1553211.3052 593292.0482 1554283.4615 594020.482799999 NaN NaN NaN NaN
﻿
Inset 4
Query: LICENCE_ID = 'A93714'
1563949.6369 596285.850199999 1565841.8245 600459.265000001 NaN NaN NaN NaN
﻿
Inset 5
Query: LICENCE_ID = 'A95414'
1547927.228 589301.7531 1550687.3752 592418.9165 NaN NaN NaN NaN
﻿
Inset 6
Query: LICENCE_ID = 'A95807'
1562034.6881 587884.325099999 1563502.1129 591489.0395 NaN NaN NaN NaN
﻿
Inset 7
Query: LICENCE_ID = 'TA0470'
1566493.0625 576279.3364 1567887.5366 579466.9352 NaN NaN NaN NaN
﻿
Inset 8
Query: LICENCE_ID = 'A93873'
1580735.5258 579412.9913 1582751.4798 580628.9197 NaN NaN NaN NaN
﻿
   ...Script Complete


In [37]:
# List all text elements in the layout
text_elements = layout.listElements("TEXT_ELEMENT")
for element in text_elements:
    print(element.name)


Text 2
Text 1
Text
Notes
Map Insets Title
Main map Key Title
Document path Text
BC Timber Sales Text
Map Title
Arrow Field team
Kootenay Business Area
Prjection etc. text
Map Prepared By Text
DSMAPBOOK - DATE
Date of Creation Text
Text 13
Text 16
Text 29
Text 27
Text 26
Text 25
Text 24
Text 21
Text 19
Text 15
Text 11
Text 10
Map Scale Text
